In [1]:
import pathlib
from tensorflow import keras

In [2]:
# Scarichiamo il dataset Flowers.
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file(origin=dataset_url, extract=True)
# Selezioniamo la cartella dove sono presenti i dati. Di solito è ~\user\username\.keras\dataset.
data_dir = pathlib.Path(data_dir).parent
data_dir = pathlib.Path(data_dir, 'flower_photos')

In [3]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
SEED = 42

# Creiamo i dataset di training e validazione. Usiamo un validation
# split di 0.2 (quindi con rapporto 80/20).
train = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED)

val = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [4]:
# Mostriamo a schermo le classi presenti nel dataset usando
# l'attributo class_names.
print(f'Classi nel dataset: {train.class_names}')

Classi nel dataset: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [5]:
# Creiamo ed addestriamo un modello.
model = keras.Sequential(
    [
        keras.Input(shape=(150, 150, 3)),
        keras.layers.Rescaling(1./255),
        keras.layers.Conv2D(
            32,
            (3, 3),
            activation='relu',
            name='conv_1'),
        keras.layers.MaxPooling2D(name='max_pool_1'),
        keras.layers.Conv2D(
            32,
            (3, 3),
            activation='relu',
            name='conv_2'),
        keras.layers.MaxPooling2D(name='max_pool_2'),
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(
            5,
            name='classification',
            activation='softmax')
    ]
)

# Copiamo il modello (ci servirà per il passo successivo)
model_callbacks = keras.models.clone_model(model)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=keras.metrics.SparseCategoricalAccuracy())

history = model.fit(
    train,
    validation_data=val,
    epochs=10)

Epoch 1/10
92/92 [==============================] - 12s 71ms/step - loss: 1.3459 - sparse_categorical_accuracy: 0.4033 - val_loss: 1.1497 - val_sparse_categorical_accuracy: 0.5150
Epoch 2/10
92/92 [==============================] - 3s 31ms/step - loss: 1.0604 - sparse_categorical_accuracy: 0.5780 - val_loss: 1.0443 - val_sparse_categorical_accuracy: 0.5681
Epoch 3/10
92/92 [==============================] - 3s 32ms/step - loss: 0.9090 - sparse_categorical_accuracy: 0.6553 - val_loss: 1.0648 - val_sparse_categorical_accuracy: 0.5913
Epoch 4/10
92/92 [==============================] - 3s 31ms/step - loss: 0.7436 - sparse_categorical_accuracy: 0.7268 - val_loss: 1.0015 - val_sparse_categorical_accuracy: 0.5967
Epoch 5/10
92/92 [==============================] - 3s 30ms/step - loss: 0.5686 - sparse_categorical_accuracy: 0.8025 - val_loss: 1.0516 - val_sparse_categorical_accuracy: 0.5940
Epoch 6/10
92/92 [==============================] - 3s 33ms/step - loss: 0.4330 - sparse_categorical_acc

In [6]:
# Riaddestriamo il modello utilizzando i callback.
model_callbacks.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=keras.metrics.SparseCategoricalAccuracy())

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        min_delta=0.01,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard()
]

history = model_callbacks.fit(
    train,
    validation_data=val,
    callbacks=callbacks,
    epochs=10)

Epoch 1/10
92/92 [==============================] - 4s 34ms/step - loss: 1.2732 - sparse_categorical_accuracy: 0.4578 - val_loss: 1.1505 - val_sparse_categorical_accuracy: 0.4973
Epoch 2/10
92/92 [==============================] - 3s 32ms/step - loss: 1.0171 - sparse_categorical_accuracy: 0.6025 - val_loss: 1.0055 - val_sparse_categorical_accuracy: 0.5736
Epoch 3/10
92/92 [==============================] - 3s 32ms/step - loss: 0.8463 - sparse_categorical_accuracy: 0.6863 - val_loss: 1.0019 - val_sparse_categorical_accuracy: 0.5954
Epoch 4/10
92/92 [==============================] - 3s 31ms/step - loss: 0.6439 - sparse_categorical_accuracy: 0.7674 - val_loss: 1.0911 - val_sparse_categorical_accuracy: 0.5831
Epoch 5/10
92/92 [==============================] - 3s 32ms/step - loss: 0.4864 - sparse_categorical_accuracy: 0.8328 - val_loss: 1.1926 - val_sparse_categorical_accuracy: 0.6022
Epoch 6/10
92/92 [==============================] - 3s 32ms/step - loss: 0.3339 - sparse_categorical_accu